# Analysis of Profitable Apps for the App Store and Google Play

As of 2018, there were about 2 million iOS apps available in the App Store, and about 2.1 million Android apps in the Google Play store. This app analysis is intended to find the most profitable types of apps to develop in terms of ad revenue, as the apps themselves will be free to download. Naturally, this means that we must find the kind of apps which are most installed, since more users means more people seeing and potentially engaging with the ads. Therefore, the goal is to help our hypothetical developers understand what kinds of apps are most likely to attract the most users on both Android and iOS.

We will accomplish these goals with a few fairly straightforward steps:

1. Open data for the App Store and Google Play apps, and create lists to more easily process the data
2. Clean the data by removing any incorrect, duplicate, or irrelevant apps (non-free, non-English, incorrect data)
3. Categorize and sort the apps so we can make informed statements about which ones users prefer

## Data exploration and sorting

Collecting data for over 4 million apps would not only be very time-consuming, it would also be expensive. Realistically, it would not even be necessary, as a smaller sample should be suitable to give us the information we need. Our data sets for the Android and iOS app stores will include about 10,000 and 7,000 apps, respectively.

Opening and exploring the data sets is the first step. We'll use a function to allow for repeated printing of rows in a readable way.

In [1]:
def explore_data(dataset, start, end, rows_columns=False):
    data_slice = dataset[start:end]
    for row in data_slice:
        print(row)
        print('\n')

    if rows_columns:
        print(f'Number of rows: {len(dataset)}')
        print(f'Number of columns: {len(dataset[0])}')

The `explore_data()` function takes four parameters: dataset, which is expected as a list of lists; start and end, which should be integers and represent the starting/ending indices of a slice of the dataset; rows_columns, expected to be a boolean with False set as default.

The function opens and slices the data. It then loops through the slice, and for each iteration prints a row and a blank line.

Let's open the two app data sets and create lists from them. We'll also save each header row into its own variable.

In [2]:
from csv import reader

### The Google Play data set ###
opened_file = open('googleplaystore.csv', encoding='UTF-8')
read_file = reader(opened_file)
android = list(read_file)
android_header = android[0]
android = android[1:]

### The App Store data set ###
opened_file = open('AppleStore.csv', encoding='UTF-8')
read_file = reader(opened_file)
ios = list(read_file)
ios_header = ios[0]
ios = ios[1:]

Simple enough! Next, we'll use our explore_data() function to examine the first few rows of the Google data file.

In [3]:
print(android_header)
print('\n')
explore_data(android, 0, 4, rows_columns=True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 10841
Number of columns: 13


There are 10,841 apps, sorted into 13 columns. Many of the column headings seem like they will be useful to our analysis. The most relevant for now are probably 'App', 'Category', 'Rating', 'Installs', 'Type', 'Price', and 'Genres'.

Now let's do the same for the iOS store data.

In [4]:
print(ios_header)
print('\n')
explore_data(ios, 0, 4, rows_columns=True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


Number of rows: 7197
Number of columns: 16


The iOS data set includes 7,197 apps with attributes sorted into 16 columns. The columns for the Apple data are in some cases a bit cryptic. Nonetheless, they are easy enough to figure out by looking at the entries. For example, track_name corresponds to the app's name. The most useful seem to be 'track_name', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', and 'prime_genre'. For additional help with the columns, the [documentation](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home) is available.

## Data Cleaning

One of the most critical parts of data analysis is cleaning the data, i.e. removing irrelevant, inaccurate, or duplicate data which would interfere with drawing accurate conclusions.

From the [discussion section](https://www.kaggle.com/lava18/google-play-store-apps/discussion) of the Google Play data, we see that row 10,472 is incorrect - it lists the app's rating as 19, while the maximum rating for an app in the Google Play Store should be 5.

In [5]:
print(android_header)  # header
print('\n')
print(android[10472])  # incorrect row
print('\n')
print(android[0])      # example correct row

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


According to the discussion, this issue is caused by a missing value for the 'Category' column. To fix this, we'll simply delete the row so it doesn't interfere with the rest of the data.

In [6]:
del android[10472] # don't run this more than once!

### Checking for Duplicates

Another issue we should check for is duplicate rows. A few duplicate entries can really skew our data and the conclusions we draw from it. Fortunately, we can check the [discussion](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/discussion) around our data; this won't always be an option so we'll also want to check for ourselves.

The easiest way to do this is create two lists to account for unique and duplicate app names:

In [7]:
android_duplicate_apps = []
android_unique_apps = []

for app in android:
    name = app[0]
    if name in android_unique_apps:
        android_duplicate_apps.append(name)
    else:
        android_unique_apps.append(name)

print('Number of duplicates: ', len(android_duplicate_apps))
print('\n')

Number of duplicates:  1181




So there are 1181 cases where an Android app name occurs more than once. Let's check the App Store as well.

In [8]:
apple_duplicate_apps = []
apple_unique_apps = []

for app in ios:
    name = app[0] # the iOS data shows the app name in the first column, labeled "ID"
    if name in apple_unique_apps:
        apple_duplicate_apps.append(name)
    else:
        apple_unique_apps.append(name)

print('Number of duplicates: ', len(apple_duplicate_apps))
print('\n')
print(apple_duplicate_apps)

Number of duplicates:  0


[]


It appears there are no duplicate entries in the App Store data.

We could just remove all the duplicate entries, but a better idea would be to examine them a little closer and remove only the entries which make sense to remove. For example, let's take a closer look at the Instagram app entries on Google Play:

In [9]:
for app in android:
    name = app[0]
    if name == 'Instagram':
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


Looking at index 3 (the fourth column), which corresponds to the number of reviews, we see that they are not all the same. This tells us that data was collected more than once for these apps at different times. With this in mind, it makes sense to only keep the most recent entry - in this case, the one with the highest count of reviews. When we remove all the duplicates, we should be left with 9659:

In [10]:
print('Expected length: ', len(android) - 1181)

Expected length:  9659


### Removing Duplicates

The safest way to remove the duplicate entries is to confirm the expected number of entries (current total - duplicates), build a new dataset of only unique values, and compare the two before deleting any rows.

First, we'll create a dictionary called `reviews_max` (specific to each store), where each dictionary key is a unique app name and the corresponding dictionary value is the highest number of reviews of that app. Then, we'll create a new data set from the dictionary, with just the latest entry per app.

In [11]:
android_reviews_max = {}

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    
    if name in android_reviews_max and android_reviews_max[name] < n_reviews:
        android_reviews_max[name] = n_reviews
        
    elif name not in android_reviews_max:
        android_reviews_max[name] = n_reviews

Now let's see if our new data set has the same length as what we expect:

In [12]:
print('Expected length:', len(android) - 1181)
print('Actual length:', len(android_reviews_max))

Expected length: 9659
Actual length: 9659


Now we can use our `reviews_max` dictionaries to remove the duplicates. As stated before, we'll only keep the entry with the most reviews for each app.

We'll start by creating two empty lists, `clean` and `already_added`. Then we can loop through the data, and add each app to the list only if the number of apps is equal to the value in our `reviews_max` dictionary, and the app doesn't already exist in the list. This will eliminate any duplicates and leave us with just the most up to date row for each unique app.

In [13]:
android_clean = []
already_added = []

for app in android:
    name = app[0]
    n_reviews = float(app[3])

    if (android_reviews_max[name] == n_reviews) and (name not in already_added):
        android_clean.append(app)
        already_added.append(name)

Let's use our `explore_data()` function to make sure everything worked as expected.

In [14]:
print('Android Data:')
print('\n')
explore_data(android_clean, 0, 3, True)

Android Data:


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9659
Number of columns: 13


Looks like we got the results we were expecting. The next step is to remove some more apps which aren't relevant to our analysis.

### Isolating English Apps

Since our company only creates apps in English, it makes sense to analyze just the English-language apps. Looking at our data, we'll find that both data sets include app names that are not in English or don't seem to target an English-speaking audience. Here are a few examples:

In [15]:
print(ios[813][1])
print(ios[6731][1])
print('\n')
print(android_clean[4412][0])
print(android_clean[7940][0])

爱奇艺PPS -《欢乐颂2》电视剧热播
【脱出ゲーム】絶対に最後までプレイしないで 〜謎解き＆ブロックパズル〜


中国語 AQリスニング
لعبة تقدر تربح DZ


We're not interested in these apps for our analysis, so let's remove them. The most logical way to do this is by comparing the characters in each app name to the characters which are typically used in English. In other words, we'll exclude apps whose names include characters which are not used in English.

Normal English characters, including letters A through Z, numbers 0 through 9, punctuation (., !, ?, ;, etc), and other symbols are each included in the ASCII standard, and has a number from 0 to 127 associated with it. With this in mind, we can check each app name to see if it includes non-English (i.e. non-ASCII) characters using a function. We'll use Python's built-in `ord()` function, which pulls a character's corresponding ASCII value.

In [16]:
def is_english(string):
    # We'll only remove an app if it has more than 3 non-ASCII characters, to minimize       unnecessary data loss.
    non_ascii = 0

    for character in string:
        if ord(character) > 127:
            non_ascii += 1
    
    if non_ascii > 3:
        return False
    else:
        return True

Let's test our function on a few app names:

In [17]:
print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Instachat 😜'))
print(is_english('Docs To Go™ Free Office Suite'))

True
False
True
True


Great! This function is somewhat simple and a few non-English apps could sneak past, but it should be mostly effective.

Next, we can apply our function to filter out the non-English apps in our data sets.

In [18]:
android_english = []
for app in android_clean:
    name = app[0]
    if is_english(name):
        android_english.append(app)

ios_english = []
for app in ios:
    name = app[1]
    if is_english(name):
        ios_english.append(app)

print('Cleaned Android apps: ', len(android_clean))
print('English Android apps: ', len(android_english))
print('Non-English Total: ', len(android_clean) - len(android_english))
print('\n')
print('Cleaned iOS apps: ', len(ios))
print('English iOS apps: ', len(ios_english))
print('Non-English Total: ', len(ios) - len(ios_english))

Cleaned Android apps:  9659
English Android apps:  9614
Non-English Total:  45


Cleaned iOS apps:  7197
English iOS apps:  6183
Non-English Total:  1014


### Isolating Free Apps

Since we are only interested in ad revenue from free apps, we'll also need to exclude any which are not free to download and install. Both data sets include both free and paid apps, so we'll have to remove the non-free apps for our analysis.

In [19]:
android_final = []
ios_final = []

for app in android_english:
    price = app[7]
    if price == '0':
        android_final.append(app)

for app in ios_english:
    price = app[4]
    if price == '0.0':
        ios_final.append(app)

print('Final number of Android apps: ', len(android_final))
print('Final number of iOS apps: ', len(ios_final))

Final number of Android apps:  8864
Final number of iOS apps:  3222


So far we've spent a good amount of time cleaning our data to make it fit our. To recap, we have:

 - Removed inaccurate data
 - Removed duplicate app entries
 - Removed non-English apps
 - Isolated the free apps

We have cut down the number of apps quite a bit, but these final app lists will give us a big enough sample size for our analysis. Now we'll move from the cleaning phase to draw some conclusions based on our data.

## Analysis

### Intro

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, we develop it further.
3. If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.


Let's consider the best approach to categorize the data. Because our end goal is to add the app on both Google Play and the App Store, we need to find app profiles that are successful on both markets. We'll start by getting an idea of the most common genres for each market (iOS and Android). We'll build frequency tables for a few key columns: `prime_genre` in the App Store, and `Genres` and `Category` in the Google Play store.

### Building the Analysis Functions

We'll use two functions for analyzing our data:

 - First, a function to generate frequency tables which include percentages
 - Second, a function to display the percentages in descending order

In [20]:
def freq_table(dataset, index):
    table = {}
    total = 0

    for row in dataset:
        total += 1
        value = row[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1

    table_percentages = {}
    for key in table:
        percentage = (table[key] / total) * 100
        table_percentages[key] = percentage

    return table_percentages

The `freq_table()` function takes two inputs, a `dataset` and an `index`, which should be an integer (the column we want to analyze). For each row in the dataset, if the specified column (in our case, the genre) is already in the frequency table, we'll add 1 to the count of that genre. If it doesn't exist yet, we'll create a new dictionary key and set it equal to 1.

Then it creates a new dictionary, converts each value into a percentage with the line `percentage = (table[key] / total) * 100`, and returns the new percentage dictionary.

In [21]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse=True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

`display_table()` embeds the `freq_table()` function we made in the previous step, and loops through each dictionary key, sorting and printing it in a nice, easily-read format.

### Apps per Genre and Category

Let's see this function in action, looking at the `prime_genre` column from our iOS data and the `Genres` and `Category` columns from the Android data.

In [22]:
display_table(ios_final, -5)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


As we can clearly see, `Games` is far and away the largest category, coming in at just over 58 percent. The next largest is `Entertainment` with 7.88 percent, `Photo & Video` with just under 5 percent. From looking at this spread, the impression is that most apps on iOS are geared toward some kind of entertainment, considering the top two categories. It gives the appearance that apps with more practical purposes are more rare, or aren't used as much. However, that does not automatically mean that games and entertainment apps have more users, as the demand for them may not match how many are being offered.

Let's continue and look at the Google Play data.

In [23]:
display_table(android_final, 1) # category column

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

At first glance, it seems to be a completely different distribution on Android. By looking at the categories near the top, most of the top categories seem to be more geared toward practical uses. Taking a closer look at the actual Play Store though, we'll quickly see that the top `FAMILY` category is mostly made up of kids games. Regardless, entertainment and "fun" apps seem to be much less in the majority in the Android ecosystem. We can confirm this further by looking at the `Genres` column:

In [24]:
display_table(android_final, -4) # prime_genre

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

Again, we see that the spread is much more evenly distributed than in the iOS store.  The `Genres` column includes a lot more catagories.

So far, the iOS market seems to be heavily biased towards games and fun, while the Android market is more balanced between both fun and practical apps. Of course, the number of apps available only tells part of the story, so next we'll take a look at the most popular apps.

### Most Popular by Category

The obvious way to determine which genres are the most popular would be to calculate the average number of installs per genre. This is easy enough for the Android data, as it includes an `Installs` column; not so much with the iOS data as there is no such column. As a workaround, we can try looking at the total number of reviews, which is found in the `rating_count_tot` column.

We'll start by calculating the average users per app genre in the iOS store. For this, we will:
- isolate the apps for each genre
- sum up the user ratings for the apps in that genre
- divide that sum by the number of apps in that genre

We can use the `freq_table()` function from the previous step to do this, using a nested `for` loop.

In [25]:
genres_ios = freq_table(ios_final,-5) # create frequency table for the prime_genre column

for genre in genres_ios:
    total = 0
    len_genre = 0
    for app in ios_final:
        genre_app = app[11]
        if genre_app == genre:
            num_ratings = float(app[5])
            total += num_ratings
            len_genre += 1

    avg_num_ratings = total / len_genre
    print(genre, ':', avg_num_ratings)

Social Networking : 71548.34905660378
Photo & Video : 28441.54375
Games : 22788.6696905016
Music : 57326.530303030304
Reference : 74942.11111111111
Health & Fitness : 23298.015384615384
Weather : 52279.892857142855
Utilities : 18684.456790123455
Travel : 28243.8
Shopping : 26919.690476190477
News : 21248.023255813954
Navigation : 86090.33333333333
Lifestyle : 16485.764705882353
Entertainment : 14029.830708661417
Food & Drink : 33333.92307692308
Sports : 23008.898550724636
Book : 39758.5
Finance : 31467.944444444445
Education : 7003.983050847458
Productivity : 21028.410714285714
Business : 7491.117647058823
Catalogs : 4004.0
Medical : 612.0


At first glance, there are a few clear leaders with a lot of reviews. Navigation, Social Networking, and Reference all have higher than average reviews compared to the others, but in reality these genres are dominated by just a few big name apps. Google Maps and Waze, Facebook and Twitter, and Bible and Dictionary.com, skew the number of reviews for their respective categories.